In [ ]:
# Testing Cell

from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.geometry.flops_based.fuselage import (
    SimpleCabinLayout,
    DetailedCabinLayout,
    BWBSimpleCabinLayout,
    BWBDetailedCabinLayout,
    FuselagePrelim,
    BWBFuselagePrelim,
)
from aviary.subsystems.geometry.flops_based.wing_detailed_bwb import (
    BWBUpdateDetailedWingDist,
    BWBComputeDetailedWingDist,
    BWBWingPrelim,
)
from aviary.subsystems.geometry.flops_based.prep_geom import (
    _Prelim,
    _BWBFuselage,
    _BWBWing,
    _Tail,
    _Fuselage,
    _FuselageRatios,
    _Wing,
)
from aviary.subsystems.geometry.flops_based.nacelle import Nacelles
from aviary.subsystems.geometry.flops_based.canard import Canard
from aviary.subsystems.geometry.flops_based.characteristic_lengths import (
    BWBWingCharacteristicLength,
    NacelleCharacteristicLength,
    OtherCharacteristicLengths,
    WingCharacteristicLength,
)
from aviary.subsystems.geometry.flops_based.wetted_area_total import TotalWettedArea
from aviary.subsystems.geometry.flops_based.wing import WingPrelim

glue_variable(get_variable_name(SimpleCabinLayout), md_code=True)
glue_variable(get_variable_name(DetailedCabinLayout), md_code=True)
glue_variable(get_variable_name(BWBSimpleCabinLayout), md_code=True)
glue_variable(get_variable_name(BWBDetailedCabinLayout), md_code=True)
glue_variable(get_variable_name(FuselagePrelim), md_code=True)
glue_variable(get_variable_name(BWBFuselagePrelim), md_code=True)

glue_variable(get_variable_name(BWBUpdateDetailedWingDist), md_code=True)
glue_variable(get_variable_name(BWBComputeDetailedWingDist), md_code=True)
glue_variable(get_variable_name(BWBWingPrelim), md_code=True)

glue_variable(get_variable_name(_Prelim), md_code=True)
glue_variable(get_variable_name(_BWBWing), md_code=True)
glue_variable(get_variable_name(_Tail), md_code=True)
glue_variable(get_variable_name(_Fuselage), md_code=True)
glue_variable(get_variable_name(_FuselageRatios), md_code=True)
glue_variable(get_variable_name(_Wing), md_code=True)
glue_variable(get_variable_name(_BWBFuselage), md_code=True)
glue_variable(get_variable_name(Nacelles), md_code=True)
glue_variable(get_variable_name(Canard), md_code=True)

glue_variable(get_variable_name(BWBWingCharacteristicLength), md_code=True)
glue_variable(get_variable_name(NacelleCharacteristicLength), md_code=True)
glue_variable(get_variable_name(OtherCharacteristicLengths), md_code=True)
glue_variable(get_variable_name(WingCharacteristicLength), md_code=True)
glue_variable(get_variable_name(TotalWettedArea), md_code=True)
glue_variable(get_variable_name(WingPrelim), md_code=True)

# FLOPS Based Blended Wing Body Modeling

The blended wing body aircraft is modeled following the FLOPS implementation. It is important to point out that the Aviary implementation is limited to certain assumptions. To model a special model, the user must read this document carefully to see it is within the scope of the Aviary implementation.

## FLOPS Based Geometry

The FLOPS based geometry subsystems are shown in the following table. The left column is for the conventional aircraft and the right column is for BWB aircraft. In both case, users can choose a simple layout or a detailed layout.

**Comparision of Conventional Aircraft and BWB Aircraft**

| Tube+Wing | BWB |
| --------- | ----- |
| {glue:md}`SimpleCabinLayout` or {glue:md}`DetailedCabinLayout`  | {glue:md}`BWBSimpleCabinLayout` or `{glue:md}BWBDetailedCabinLayout` |
|   | {glue:md}`BWBUpdateDetailedWingDist` or {glue:md}`BWBComputeDetailedWingDist` |
| {glue:md}`FuselagePrelim` | {glue:md}`BWBFuselagePrelim` |
| {glue:md}`WingPrelim` | {glue:md}`BWBWingPrelim` |
| {glue:md}`_Prelim` | {glue:md}`_Prelim` |
| {glue:md}`_Wing` | {glue:md}`_BWBWing` |
| {glue:md}`_Tail` | {glue:md}`_Tail` |
| {glue:md}`_Fuselage` | {glue:md}`_BWBFuselage` |
| {glue:md}`_FuselageRatios` | {glue:md}`_FuselageRatios` |
| {glue:md}`Nacelles` | {glue:md}`Nacelles` |
| {glue:md}`Canard` | {glue:md}`Canard` |
| {glue:md}`WingCharacteristicLength` | {glue:md}`BWBWingCharacteristicLength` |
| {glue:md}`NacelleCharacteristicLength` | {glue:md}`NacelleCharacteristicLength` |
| {glue:md}`OtherCharacteristicLengths` | {glue:md}`OtherCharacteristicLengths` |
| {glue:md}`TotalWettedArea` | {glue:md}`TotalWettedArea` |

As we see in the above table, some geometrical components have no special code in the BWB case (e.g. {glue:md}`Nacelles` and {glue:md}`_Tail`). So, the new implementation of BWB geometry uses the same components in conventional aircraft.

Comparing to traditional tube and wing model, Blended wing body (BWB) modeling has two major new changes in geometry subsystems:

- Wings always have detailed wing data,
- Fuselage layout is computed using a special algorithm.

We will explain some details of each feature in this document.

In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_all_non_aviary_names, get_variable_name, glue_variable

# new variables added for FLOPS based BWB geometry
glue_variable(
    get_variable_name(Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_BUSINESS), md_code=True
)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_FIRST), md_code=True)
glue_variable(
    get_variable_name(Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_ECONOMY), md_code=True
)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.SEAT_PITCH_BUSINESS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.SEAT_PITCH_FIRST), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.SEAT_PITCH_ECONOMY), md_code=True)

glue_variable(get_variable_name(Aircraft.BWB.NUM_BAYS), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuselage.SIMPLE_LAYOUT), md_code=True)

Rear_spar_percent_chord = get_all_non_aviary_names(BWBFuselagePrelim, include_in_out='in')[0]
glue_variable('Rear_spar_percent_chord', Rear_spar_percent_chord, md_code=True)

### Fuselage Layout

There are two options: simple layout and detailed layout. For simple layout, you are given fuselage length, width and leading edge angle (as well as the rear spar percent chord at fuselage centerline, default to 70%), Aviary computes passenger compartment length, wing root chord, cabin area. This is shown in the following image:

![Fuselage size](../images/BWB_FLOPS_Fuselage_Simple_Geom.png)

Variable {glue:md}`Rear_spar_percent_chord` serves as the ratio of fuselage length and compartment length. It says that the rear spar is mounted at the side of fuselage which is 70% (default) of fuselage length from the leading edge. {glue:md}`Rear_spar_percent_chord` is hard coded for now, but can be easily converted to an Aviary variable.

The detailed layout involves more details like the number of passengers, seat pitch and seat abreast. Fuselage length and width are no longer inputs but outputs. If you have those data, it is a better choice.

For BWB, there is a new concept: the number of passenger cabin bays (or {glue:md}`Aircraft.BWB.NUM_BAYS`). It is a discrete, pressurized structural compartment within the BWB center body that contains passenger seating and is bounded by primary load-carrying frames, ribs, and pressure bulkheads. In Aviary, we determine the number of bays as how many blocks can be fit within the fuselage cabin width, assuming each block has a 12-foot width (hard coded). Note that it is not a layout term and bays are not separated by aisles even though the number of bays is computed here in fuselage layout components. The following is an example of six bays.

![Fuselage size](../images/BWB_FLOPS_fuselage_3_Bays.png)

Finally, fuselage height is computed from the width and height-to-width ratio.

To choose detailed layout or simple layout, the user can set up a flag {glue:md}`Aircraft.Fuselage.SIMPLE_LAYOUT`. If it is true, Aviary assumes a simple layout. Otherwise, it is a detailed layout.

In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_all_non_aviary_names, get_variable_name, glue_variable

glue_variable(get_variable_name(Aircraft.Wing.INPUT_STATION_DIST), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.CHORD_PER_SEMISPAN_DIST), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.THICKNESS_TO_CHORD_DIST), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.LOAD_PATH_SWEEP_DIST), md_code=True)

BWB_CHORD_PER_SEMISPAN_DIST = get_all_non_aviary_names(
    BWBUpdateDetailedWingDist, include_in_out='out'
)[0]
glue_variable('BWB_CHORD_PER_SEMISPAN_DIST', BWB_CHORD_PER_SEMISPAN_DIST, md_code=True)
BWB_THICKNESS_TO_CHORD_DIST = get_all_non_aviary_names(
    BWBUpdateDetailedWingDist, include_in_out='out'
)[1]
glue_variable('BWB_THICKNESS_TO_CHORD_DIST', BWB_THICKNESS_TO_CHORD_DIST, md_code=True)
BWB_LOAD_PATH_SWEEP_DIST = get_all_non_aviary_names(
    BWBUpdateDetailedWingDist, include_in_out='out'
)[2]
glue_variable('BWB_LOAD_PATH_SWEEP_DIST', BWB_LOAD_PATH_SWEEP_DIST, md_code=True)

### Wings

#### Detailed Wings

For the BWB, the wing weight is always calculated with detailed wing geometry. The wing station distribution is either given ({glue:md}`BWBUpdateDetailedWingDist`) or created ({glue:md}`BWBComputeDetailedWingDist`). This means that number of input wing stations is always positive. If input wing stations are not provided, FLOPS adds three points. We call this case the simple wing case. Otherwise, we say that we have a detailed wing case.

Let's assume the detailed wing data is given. That means we are given the following arrays:

- {glue:md}`Aircraft.Wing.INPUT_STATION_DIST`
- {glue:md}`Aircraft.Wing.CHORD_PER_SEMISPAN_DIST`
- {glue:md}`Aircraft.Wing.THICKNESS_TO_CHORD_DIST`
- {glue:md}`Aircraft.Wing.LOAD_PATH_SWEEP_DIST`

These data are used to define an outboard wing panel to be added to the side of the cabin. In FLOPS, data start from the wing root to the wing tip. For transporter aircraft, it is good enough. But for BWB aircraft, we need to add one additional wing station at the body centerline. So, if you obtain the data from a FLOPS data set. You need to add one entry at the beginning of the lists. 

{glue:md}`Aircraft.Wing.INPUT_STATION_DIST` is the input station distribution (an Aviary option). Each entry is a location of an input station, either as a fraction from wing root to wing tip (when value < 1.0), or the absolute location. If you obtain the data from a FLOPS input data, add a 0.0 at the beginning.

{glue:md}`Aircraft.Wing.CHORD_PER_SEMISPAN_DIST` is the chord length, either as a fraction of semispan or actual chord. If you obtain the data from a FLOPS input data, add a -1.0 at the beginning. It will be updated during the run.

{glue:md}`Aircraft.Wing.THICKNESS_TO_CHORD_DIST` is the thickness to chord ratios at the input station distribution. If you obtain the data from a FLOPS input data, add a -1.0 at the beginning. It will be updated during the run.

{glue:md}`Aircraft.Wing.LOAD_PATH_SWEEP_DIST` is the load path sweep angle between two adjacent input station. f you obtain the data from a FLOPS input data, add a 0.0 at the beginning. Note that this list should have one fewer entry than the other three lists.

When Aviary runs, {glue:md}`Aircraft.Wing.CHORD_PER_SEMISPAN_DIST`, {glue:md}`Aircraft.Wing.THICKNESS_TO_CHORD_DIST`, and {glue:md}`Aircraft.Wing.LOAD_PATH_SWEEP_DIST` are updated and output to local variabls {glue:md}`BWB_THICKNESS_TO_CHORD_DIST`, {glue:md}`BWB_THICKNESS_TO_CHORD_DIST`, and {glue:md}`BWB_LOAD_PATH_SWEEP_DIST`.

#### Simple Wings

Keep in mind that the BWB aircraft always have detailed wings, but users do not have to provide the data as above. In this case, the user should provide (0.0, 0.5, 1.0) as {glue:md}`Aircraft.Wing.INPUT_STATION_DIST`. Here, `Aircraft.Wing.INPUT_STATION_DIST[0] = 0.0` means that the first station is at the fuselage center line （at 0.0%). `Aircraft.Wing.INPUT_STATION_DIST[2] = 1.0` means that the last station is at the tip (namely 100% place). `Aircraft.Wing.INPUT_STATION_DIST[1] = 0.5` is a temporary place holder. When Aviary runs, `Aircraft.Wing.INPUT_STATION_DIST[1]` is updated to the location of root chord (note that this is the actual location, not the percentage). The same three local variables  `BWB_THICKNESS_TO_CHORD_DIST` (dimension 3), {glue:md}`BWB_THICKNESS_TO_CHORD_DIST` (dimension 3), and {glue:md}`BWB_LOAD_PATH_SWEEP_DIST` (dimension 2) are updated accordingly. See {glue:md}`BWBComputeDetailedWingDist` component for details.

In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.geometry.flops_based.characteristic_lengths import (
    HorizontalTailCharacteristicLength,
    VerticalTailCharacteristicLength,
)

glue_variable(get_variable_name(Aircraft.BWB.DETAILED_WING_PROVIDED), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuselage.CROSS_SECTION), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuselage.WETTED_AREA), md_code=True)

glue_variable(get_variable_name(Aircraft.HorizontalTail.CHARACTERISTIC_LENGTH), md_code=True)
glue_variable(get_variable_name(Aircraft.VerticalTail.CHARACTERISTIC_LENGTH), md_code=True)

glue_variable(get_variable_name(HorizontalTailCharacteristicLength), md_code=True)
glue_variable(get_variable_name(VerticalTailCharacteristicLength), md_code=True)

### Fuselage and Wing

As summary, the geometry subsystem for BWB aircraft is very similar to that of transporter aircraft. See the diagram below:

![Fuselage size](../images/BWB_FLOPS_geom.png)

To choose between simple fuselage layout or detailed fuselage layout, the user sets {glue:md}`Aircraft.Fuselage.SIMPLE_LAYOUT` to True or False. For BWB, to choose between providing detailed wing or letting Aviary to create a detailed wing, the user sets {glue:md}`Aircraft.BWB.DETAILED_WING_PROVIDED` to True or False.

Comparing the two cases （transporter and BWB), we see that {glue:md}`BWBComputeDetailedWingDist` and {glue:md}`BWBUpdateDetailedWingDist` are new. This is because we must update {glue:md}`Aircraft.Wing.CHORD_PER_SEMISPAN_DIST`, {glue:md}`Aircraft.Wing.THICKNESS_TO_CHORD_DIST`, and {glue:md}`Aircraft.Wing.LOAD_PATH_SWEEP_DIST` in the case of BWB.

This feature is explained in the following diagram (in the case of detailed wing provided):

![Fuselage size](../images/BWB_FLOPS_Geom_fus_wing.png)

### Fuselage Cross Section and Wetted Area

Because of the nature of blended wing body, we ignore the fuselage wetted area. There, both {glue:md}`Aircraft.Fuselage.CROSS_SECTION` and {glue:md}`Aircraft.Fuselage.WETTED_AREA` are set to 0.0.

This is implemented in {glue:md}`_BWBFuselage`.

### Characteristics

The FLOPS based BWB aircraft does not have horizontal and vertical tails. Once again ,we simply set both {glue:md}`Aircraft.HorizontalTail.CHARACTERISTIC_LENGTH` and {glue:md}`Aircraft.VerticalTail.CHARACTERISTIC_LENGTH` to 0.0.

This is because the FLOPS algorithm does not apply to non-horizontal and non-vertical tails scenarios. If users want to add them, their mass equations must be provided separately. See {glue:md}`HorizontalTailCharacteristicLength` and {glue:md}`VerticalTailCharacteristicLength` classes for details.

In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.mass.flops_based.furnishings import BWBFurnishingsGroupMass
from aviary.subsystems.mass.flops_based.fuselage import BWBFuselageMass, BWBAftBodyMass
from aviary.subsystems.mass.flops_based.wing_common import BWBWingMiscMass
from aviary.subsystems.mass.flops_based.wing_detailed import BWBDetailedWingBendingFact

glue_variable(get_variable_name(BWBFurnishingsGroupMass), md_code=True)
glue_variable(get_variable_name(BWBFuselageMass), md_code=True)
glue_variable(get_variable_name(BWBAftBodyMass), md_code=True)
glue_variable(get_variable_name(BWBWingMiscMass), md_code=True)
glue_variable(get_variable_name(BWBDetailedWingBendingFact), md_code=True)

glue_variable(get_variable_name(Aircraft.Furnishings.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuselage.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.BWB_AFTBODY_MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.MISC_MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.BENDING_MATERIAL_FACTOR), md_code=True)

## FLOPS Based Mass

Aviary computes masses of indiviual components and then sum them up. For BWB aircrafts, most of them share the same code with transporters. There are a few exceptions, however. They are:

- {glue:md}`BWBFurnishingsGroupMass` (computation of {glue:md}`Aircraft.Furnishings.MASS`)
- {glue:md}`BWBFuselageMass` (computation of {glue:md}`Aircraft.Fuselage.MASS`)
- {glue:md}`BWBAftBodyMass` (computation of {glue:md}`Aircraft.Wing.BWB_AFTBODY_MASS`)
- {glue:md}`BWBWingMiscMass` (computation of {glue:md}`Aircraft.Wing.MISC_MASS`)
- {glue:md}`BWBDetailedWingBendingFact` (computation of {glue:md}`Aircraft.Wing.BENDING_MATERIAL_FACTOR`)

Here, {glue:md}`BWBAftBodyMass` is a new component for BWB. This area starts from the rear span and backward.

Because the wing is always a detailed wing for BWB aircraft, we must compute the wing bending material factor with sweep adjustment. This is done in {glue:md}`BWBDetailedWingBendingFact`.

## FLOPS Based Aerodynamics

There is no special implementation of aerodynamics subsystem in FLOPS. It is likely that users need to provide their own aero tables for their design.